In [ ]:
import numpy as np
from math import factorial
from cmath import exp


def simplex_content(j, n_dims, signed, *args):
    """
    Compute simplex content (j-dim-volume) using Cayley-Menger Determinant
    :param j: dimension of simplex
    :param n_dims: dimension of R^n space
    :param signed: bool denoting whether to calculate signed content or unsigned content
    :param args: v0, v1, ... vectors for the coordinates of the vertices defining the simplex

    :return: vol: volume of the simplex
    """
    assert(n_dims == len(args[0]))
    assert(isinstance(signed, bool))
    n_vert = len(args)
    assert(n_vert == j+1)

    if n_dims > j:
        assert(not signed)

    if not signed:
        # construct Cayley-Menger matrix
        B = np.zeros([j+2, j+2])
        B[:, 0] = 1
        B[0, :] = 1
        B[0, 0] = 0
        for r in range(1, j+2):
            for c in range(r+1, j+2):
                vr = args[r-1]
                vc = args[c-1]
                B[r, c] = np.linalg.norm(vr-vc) ** 2
                B[c, r] = B[r, c]
        vol2 = (-1)**(j+1) / (2**j) / (factorial(j)**2) * np.linalg.det(B)
        if vol2 < 0:
            print("Warning: zeroing small negative number {0}".format(vol2))
        vol = np.sqrt(max(0, vol2))
    else:
        # matrix determinant
        mat = np.zeros([j, j])
        for r in range(j):
            mat[:, r] = args[r] - args[-1]
        vol = np.linalg.det(mat) / factorial(j)

    return vol

def simplex_ft_bw_cpu(V, E, res=(256, 256), j=2):
    assert(n_dims == len(res))  # consistent spacial dimensionality
    assert(E.shape[0] == D.shape[0])  # consistent vertex numbers
    assert(mode in ['density', 'mass'])

#     add noise for enhanced robustness
#     V += eps * np.random.rand(*V.shape)

    # number of columns in E
    ghost = E.shape[1] == j and n_dims == j
    assert (E.shape[1] == j+1 or ghost)
    if ghost:
        V = np.append(V, np.zeros([1, n_dims]), axis=0)
        E = np.append(E, V.shape[0] - 1 + np.zeros([E.shape[0], 1], dtype=np.int), axis=1)
    n_elem = E.shape[0]
    n_vert = V.shape[0]
    n_channel = D.shape[1]

    # frequency tensor
    omega = fftfreqs(res)
    omega[tuple([0] * n_dims)] += 1e-3 # will get rid of this

    # normalize frequencies
    for dim in range(n_dims):
        omega[..., dim] *= 2 * np.pi / t[dim]
    
    for ie in range(E.shape[0]):
        verts = V[E[ie]]
        sig_list = []
        esig_list = []
        for vert in verts:
            sig = np.sum(V[vert] * omega, axis=-1)
            esig = np.exp(-1j * sig)
            sig_list.append(sig)
            esig_list.append(esig)
        for i in range(j+1):
            

In [ ]:
from time import time

V = np.array([[0.2,0.2],
              [0.8,0.6],
              [0.8,0.8],
              [0.6,0.8]])
V = V+1e-10*np.random.rand(*V.shape)
E = np.array([[0,1,2],
              [2,3,0]])
D = np.ones((E.shape[0], 1))
